In [4]:
# Importing the required dependencies or libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly
import plotly.express as px
import plotly.offline as pyo
import cufflinks as cf
import sklearn.utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
pyo.init_notebook_mode(connected=True)
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
cf.go_offline()
%matplotlib inline

In [5]:
grid_data = pd.read_csv('./Data/Data_for_UCI_named.csv')

In [6]:
grid_data.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [7]:
grid_data.tail()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
9995,2.930406,9.487627,2.376523,6.187797,3.343416,-0.658054,-1.449106,-1.236256,0.601709,0.779642,0.813512,0.608385,0.023892,unstable
9996,3.392299,1.274827,2.954947,6.894759,4.349512,-1.663661,-0.952437,-1.733414,0.502079,0.567242,0.285880,0.366120,-0.025803,stable
9997,2.364034,2.842030,8.776391,1.008906,4.299976,-1.380719,-0.943884,-1.975373,0.487838,0.986505,0.149286,0.145984,-0.031810,stable
9998,9.631511,3.994398,2.757071,7.821347,2.514755,-0.966330,-0.649915,-0.898510,0.365246,0.587558,0.889118,0.818391,0.037789,unstable
9999,6.530527,6.781790,4.349695,8.673138,3.492807,-1.390285,-1.532193,-0.570329,0.073056,0.505441,0.378761,0.942631,0.045263,unstable


In [8]:
grid_data.shape

(10000, 14)

In [11]:
grid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [9]:
grid_data.describe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878
max,9.999469,9.999837,9.999450,9.999443,5.864418,-0.500108,-0.500072,-0.500025,0.999937,0.999944,0.999982,0.999930,0.109403


In [13]:
grid_data.isnull().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [14]:
grid_data.duplicated().sum()

0

In [19]:
features = grid_data.drop(['stab', 'stabf'], axis = 1)
target = grid_data['stabf']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=1)

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (8000, 12)
y_train shape: (8000,)
X_test shape: (2000, 12)
y_test shape: (2000,)


In [22]:
scaler = StandardScaler()
scaler.fit(X_train)

x_train_scaled = scaler.transform(X_train)
x_test_scaled = scaler.transform(X_test)

x_train = pd.DataFrame(x_train_scaled, columns = X_train.columns)
x_test = pd.DataFrame(x_test_scaled, columns = X_test.columns)

In [23]:
random_forest = RandomForestClassifier(random_state=1)
random_forest.fit(X_train, y_train)
y_test_predictions = random_forest.predict(X_test)

In [39]:
accuracy = accuracy_score(y_test, y_test_predictions)
print('Accuracy: {}'.format(accuracy))

precision = precision_score(y_test, y_test_predictions, pos_label='stable')
print('Precision: {}'.format(round(precision * 100), 2))

recall = recall_score(y_test, y_test_predictions, pos_label='stable')
print('Recall: {}'.format(round(recall * 100), 2))

f1 = f1_score(y_test, y_test_predictions, pos_label='stable')
print('F1 Score: {}'.format(round(f1 * 100), 2))

Accuracy: 0.929
Precision: 92
Recall: 88
F1 Score: 90


In [27]:
print('Classification Report:\n',
      classification_report(y_test, y_test_predictions, digits=4))

Classification Report:
               precision    recall  f1-score   support

      stable     0.9191    0.8778    0.8980       712
    unstable     0.9341    0.9573    0.9456      1288

    accuracy                         0.9290      2000
   macro avg     0.9266    0.9176    0.9218      2000
weighted avg     0.9288    0.9290    0.9286      2000



In [28]:
random_forest_cnf_mat = confusion_matrix(y_test,
                              y_test_predictions,
                              labels=['unstable', 'stable'])
print('Confusion Matrix:\n', random_forest_cnf_mat)

Confusion Matrix:
 [[1233   55]
 [  87  625]]


In [32]:
xgb = XGBClassifier(random_state=1)

xgb.fit(x_train_scaled, y_train)

xgb_pred = xgb.predict(x_test_scaled)

[23:38:23] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [37]:
xgb_accuracy = accuracy_score(y_test, xgb_pred)
print('Accuracy: {}'.format(xgb_accuracy))

xgb_precision = precision_score(y_test, xgb_pred, pos_label='stable')
print('Precision: {}'.format(round(xgb_precision * 100), 2))

xgb_recall = recall_score(y_test, xgb_pred, pos_label='stable')
print('Recall: {}'.format(round(xgb_recall * 100), 2))

xgb_f1 = f1_score(y_test, xgb_pred, pos_label='stable')
print('F1: {}'.format(round(xgb_f1 * 100), 2))

Accuracy: 0.9455
Precision: 94
Recall: 91
F1: 92


In [42]:
lgbm_clas = LGBMClassifier(random_state=1)
lgbm_clas.fit(X_train, y_train)
lgbm_pred = lgbm_clas.predict(X_test)

In [43]:
lgbm_cnf_mat = confusion_matrix(y_true=y_test,
                           y_pred=lgbm_pred,
                           labels=['stable', 'unstable'])

lgbm_cnf_mat

array([[ 634,   78],
       [  44, 1244]], dtype=int64)

In [44]:
lgbm_accuracy = accuracy_score(y_test, lgbm_pred)
print('Accuracy: {}'.format(lgbm_accuracy))

Accuracy: 0.939


In [45]:
print(classification_report(y_test, lgbm_pred, digits=4))

              precision    recall  f1-score   support

      stable     0.9351    0.8904    0.9122       712
    unstable     0.9410    0.9658    0.9533      1288

    accuracy                         0.9390      2000
   macro avg     0.9381    0.9281    0.9327      2000
weighted avg     0.9389    0.9390    0.9387      2000



In [46]:
extra_trees = ExtraTreesClassifier(random_state=1)
extra_trees.fit(X_train, y_train)
extra_trees = extra_trees.predict(X_test)

In [47]:
extra_trees_cnf_mat = confusion_matrix(y_true=y_test,
                                       y_pred=extra_trees,
                                       labels=['stable', 'unstable'])
extra_trees_cnf_mat

TypeError: Expected sequence or array-like, got <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>

In [48]:
print(classification_report(y_test, extra_trees, digits=4))

TypeError: Expected sequence or array-like, got <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>

In [49]:
extra_trees_parameters = {
    'n_estimators': [50, 100, 300, 500, 1000],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'min_samples_split': [2, 3, 5, 7, 9],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

In [50]:
extra_trees_rscv = RandomizedSearchCV(
    estimator=extra_trees,
    param_distributions=extra_trees_parameters,
    cv=5,
    n_iter=10,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1,
    random_state=1)

In [51]:
extra_trees_rscv.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=ExtraTreesClassifier(random_state=1),
                   n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 3, 5, 7, 9],
                                        'n_estimators': [50, 100, 300, 500,
                                                         1000]},
                   random_state=1, scoring='accuracy', verbose=1)

In [52]:
extra_trees_rscv.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

In [53]:
rscv_pred = extra_trees_rscv.predict(X_test)

In [54]:
extra_trees_rscv_cfn_mat = confusion_matrix(y_true=y_test,
                               y_pred=rscv_pred,
                               labels=['stable', 'unstable'])

extra_trees_rscv_cfn_mat

array([[ 619,   93],
       [  53, 1235]], dtype=int64)

In [55]:
print(classification_report(y_test, rscv_pred, digits= 4))

              precision    recall  f1-score   support

      stable     0.9211    0.8694    0.8945       712
    unstable     0.9300    0.9589    0.9442      1288

    accuracy                         0.9270      2000
   macro avg     0.9256    0.9141    0.9193      2000
weighted avg     0.9268    0.9270    0.9265      2000



In [56]:
print(extra_trees.feature_importances_)

[0.11739736 0.11844468 0.11316851 0.11546569 0.03950675 0.04037132
 0.04070628 0.04057864 0.08978291 0.09367636 0.09688268 0.09401882]
